# Chapter 9: Using the Session

Postponed Evaluation of Annotations:

In [359]:
from __future__ import annotations

Necessary imports:

In [360]:
from sqlalchemy import create_engine

In [361]:
from sqlalchemy.orm import sessionmaker

We will use memory storage for this chapter:

In [362]:
DATABASE_URL = "sqlite+pysqlite:///:memory:"

Engine:

In [363]:
engine = create_engine(
    DATABASE_URL,
    echo=True,
)

Configuring a session factory:

In [364]:
SessionMaker = sessionmaker(
    bind=engine,
    expire_on_commit=True,
    autoflush=True,
)

Imports and re-usable types:

In [365]:
import datetime
from typing import Annotated

from sqlalchemy import (CheckConstraint, ForeignKey, String)
from sqlalchemy.orm import (DeclarativeBase, Mapped, MappedAsDataclass,
                            mapped_column, relationship,)

In [366]:
int_pk = Annotated[
    int,
    mapped_column(
        primary_key=True,
    )
]
date_auto = Annotated[
    datetime.date,
    mapped_column(
        default=datetime.date.today,
    )
]
str_127 = Annotated[
    str,
    mapped_column(
        String(127),
    )
]

Declarative base class with dataclass integration:

In [367]:
class Base(MappedAsDataclass, DeclarativeBase):
    pass

Employee model:

In [368]:
class Employee(Base):
    __tablename__ = "employee"

    employee_id: Mapped[int_pk] = mapped_column(init=False)

    manager_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    name: Mapped[str_127] = mapped_column(
        CheckConstraint(
            "length(name)>0",
            name="name_length_must_be_at_least_one_character",
        ),
        default="",
    )
    is_manager: Mapped[bool] = mapped_column(default=False)
    hire_date: Mapped[date_auto] = mapped_column(default=None)

    # self-referential relationship: manager/employees
    manager: Mapped[Employee] = relationship(
        back_populates="employees",
        remote_side=[employee_id],
        init=False,
        repr=False,
    )
    employees: Mapped[list[Employee]] = relationship(
        back_populates="manager",
        init=False,
        repr=False,
    )

In [369]:
Base.metadata.create_all(engine)

2024-03-26 11:18:24,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 11:18:24,633 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-26 11:18:24,634 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-26 11:18:24,635 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2024-03-26 11:18:24,635 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-26 11:18:24,636 INFO sqlalchemy.engine.Engine 
CREATE TABLE employee (
	employee_id INTEGER NOT NULL, 
	manager_id INTEGER, 
	name VARCHAR(127) NOT NULL CONSTRAINT name_length_must_be_at_least_one_character CHECK (length(name)>0), 
	is_manager BOOLEAN NOT NULL, 
	hire_date DATE NOT NULL, 
	PRIMARY KEY (employee_id), 
	FOREIGN KEY(manager_id) REFERENCES employee (employee_id)
)


2024-03-26 11:18:24,637 INFO sqlalchemy.engine.Engine [no key 0.00033s] ()
2024-03-26 11:18:24,637 INFO sqlalchemy.engine.Engine COMMIT


## Object States in a Session

In [370]:
from sqlalchemy import inspect

In [371]:
session = SessionMaker()

A new ORM object starts in the **transient** state:

In [372]:
nobody = Employee(name="Nobody")

In [373]:
print("STATE: transient", inspect(nobody).transient)  # True

STATE: transient True


Adding this object to the session using the `Session.add()` method changes its state to **pending**:

In [374]:
session.add(nobody)

In [375]:
print("STATE: pending", inspect(nobody).pending)  # True

STATE: pending True


We can inspect `Session.new` to view all "new" instances within this session:


In [376]:
print(session.new)

IdentitySet([Employee(employee_id=None, manager_id=None, name='Nobody', is_manager=False, hire_date=None)])


In [377]:
print("Object in session:", nobody in session)  # True

Object in session: True


To persist this instance, invoke `Session.flush()`. This changes the object state to **persistent**:

In [378]:
session.flush()

2024-03-26 11:18:24,701 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 11:18:24,703 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-26 11:18:24,703 INFO sqlalchemy.engine.Engine [generated in 0.00063s] (None, 'Nobody', 0, '2024-03-26')


In [379]:
print("STATE: persistent", inspect(nobody).persistent) # True

STATE: persistent True


A primary key is generated and can be used to retrieve an instance using `Session.get()`:

In [380]:
print(session.get(Employee, nobody.employee_id))

Employee(employee_id=1, manager_id=None, name='Nobody', is_manager=False, hire_date=datetime.date(2024, 3, 26))


When you delete an instance and "flush", the instance state transitions to **deleted**:

In [381]:
session.delete(nobody)

In [382]:
session.flush()

2024-03-26 11:18:24,722 INFO sqlalchemy.engine.Engine SELECT employee.employee_id AS employee_employee_id, employee.manager_id AS employee_manager_id, employee.name AS employee_name, employee.is_manager AS employee_is_manager, employee.hire_date AS employee_hire_date 
FROM employee 
WHERE ? = employee.manager_id
2024-03-26 11:18:24,723 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (1,)
2024-03-26 11:18:24,726 INFO sqlalchemy.engine.Engine DELETE FROM employee WHERE employee.employee_id = ?
2024-03-26 11:18:24,726 INFO sqlalchemy.engine.Engine [generated in 0.00039s] (1,)


In [383]:
print("STATE: deleted", inspect(nobody).deleted)  # True

STATE: deleted True


After you commit the current transaction and close the session, the instance enters the **detached** state:

In [384]:
session.commit()

2024-03-26 11:18:24,734 INFO sqlalchemy.engine.Engine COMMIT


In [385]:
session.close()

In [386]:
print("STATE: detached", inspect(nobody).detached)  # True

STATE: detached True


## Commit styles

Dummy employee data:

In [387]:
employee1 = Employee(name="One")

In [388]:
employee2 = Employee(name="Two")

In [389]:
employee3 = Employee(name="Three")

commit-as-you-go:

In [390]:
with SessionMaker() as session:
    session.add(employee1)
    session.commit()  # commit as often as you need

    session.add(employee2)
    session.commit()  # commit as often as you need

2024-03-26 11:18:24,756 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 11:18:24,757 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-26 11:18:24,757 INFO sqlalchemy.engine.Engine [cached since 0.055s ago] (None, 'One', 0, '2024-03-26')
2024-03-26 11:18:24,758 INFO sqlalchemy.engine.Engine COMMIT
2024-03-26 11:18:24,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 11:18:24,759 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-26 11:18:24,760 INFO sqlalchemy.engine.Engine [cached since 0.05707s ago] (None, 'Two', 0, '2024-03-26')
2024-03-26 11:18:24,760 INFO sqlalchemy.engine.Engine COMMIT


begin-once:

In [391]:
with SessionMaker.begin() as session:
    session.add(employee3)
# commits transaction and closes session

2024-03-26 11:18:24,764 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 11:18:24,765 INFO sqlalchemy.engine.Engine INSERT INTO employee (manager_id, name, is_manager, hire_date) VALUES (?, ?, ?, ?)
2024-03-26 11:18:24,765 INFO sqlalchemy.engine.Engine [cached since 0.06244s ago] (None, 'Three', 0, '2024-03-26')
2024-03-26 11:18:24,766 INFO sqlalchemy.engine.Engine COMMIT


Let's check what has been written to the employee table:

In [392]:
from sqlalchemy import select

In [393]:
stmt = select(Employee)

In [394]:
with SessionMaker() as session:
    for employee in session.scalars(stmt):
        print(employee)

2024-03-26 11:18:24,776 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-26 11:18:24,777 INFO sqlalchemy.engine.Engine SELECT employee.employee_id, employee.manager_id, employee.name, employee.is_manager, employee.hire_date 
FROM employee
2024-03-26 11:18:24,777 INFO sqlalchemy.engine.Engine [generated in 0.00038s] ()
Employee(employee_id=1, manager_id=None, name='One', is_manager=False, hire_date=datetime.date(2024, 3, 26))
Employee(employee_id=2, manager_id=None, name='Two', is_manager=False, hire_date=datetime.date(2024, 3, 26))
Employee(employee_id=3, manager_id=None, name='Three', is_manager=False, hire_date=datetime.date(2024, 3, 26))
2024-03-26 11:18:24,778 INFO sqlalchemy.engine.Engine ROLLBACK
